## Query source LH, save to target LH + adjust to semantic model
This notebook queries the source lakehouse based on a defined SQL query. After that, it saves the result set to the target lakehouse. 

Next, it uses Semantic Link Labs to add the resulting table from the target lakehouse to the defined semantic model. 

In [1]:
%pip install semantic-link-labs
import sempy_labs as labs
import sempy.fabric as fabric

StatementMeta(, 85c6cecf-e197-4edd-bb46-37d3121e82b8, 8, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.5/713.5 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 164.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.9.3
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-ff0c9466-9a54-40a6-acb1-d872859d7e40
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
# Define source and target lakehouses (schemas/databases)
source_lakehouse = "LH_STORE_Silver"
target_lakehouse = "LH_STORE_Gold"

StatementMeta(, 21901fc9-9758-4753-82e6-e88f50f50d90, 10, Finished, Available, Finished)

#### Create table in destination lakehouse

In [2]:
# Define source table and target table names
source_table = "internetsales"
target_table = "sales"  # new table in Gold to save results

StatementMeta(, 85c6cecf-e197-4edd-bb46-37d3121e82b8, 10, Finished, Available, Finished)

In [5]:
sql_query = f"""
SELECT 
    SalesOrderNumber AS OrderNumber,
    ProductKey, 
    OrderDateKey, 
    ShipDateKey,
    CustomerKey,
    OrderQuantity AS Quantity,
    SalesAmount AS Price
FROM {source_lakehouse}.{source_table}
"""

# Run the SQL query and get a DataFrame
df = spark.sql(sql_query)

# Show first 5 rows of the resulting DataFrame
df.show(5)


StatementMeta(, fc4b98e5-69eb-41d3-a6d2-bfbb71cc3aa6, 13, Finished, Available, Finished)

+-----------+----------+------------+-----------+-----------+--------+--------------------+
|OrderNumber|ProductKey|OrderDateKey|ShipDateKey|CustomerKey|Quantity|               Price|
+-----------+----------+------------+-----------+-----------+--------+--------------------+
|    SO51555|       480|    20190623|   20190630|      11037|       1|2.290000000000000000|
|    SO58176|       480|    20191018|   20191025|      14338|       1|2.290000000000000000|
|    SO64622|       480|    20200121|   20200128|      11951|       1|2.290000000000000000|
|    SO66652|       480|    20200220|   20200227|      16329|       1|2.290000000000000000|
|    SO69217|       480|    20200328|   20200404|      16147|       1|2.290000000000000000|
+-----------+----------+------------+-----------+-----------+--------+--------------------+
only showing top 5 rows



In [6]:
# Write the DataFrame as a new table in the target lakehouse
df.write.mode("overwrite").saveAsTable(f"{target_lakehouse}.{target_table}")

print(f"✅ Saved filtered data to table '{target_table}' in lakehouse '{target_lakehouse}'")

StatementMeta(, fc4b98e5-69eb-41d3-a6d2-bfbb71cc3aa6, 14, Finished, Available, Finished)

✅ Saved filtered data to table 'sales' in lakehouse 'LH_STORE_Gold'


#### Update Semantic Model

In [3]:
semanticmodel_name = "Sales Analysis"

StatementMeta(, 85c6cecf-e197-4edd-bb46-37d3121e82b8, 11, Finished, Available, Finished)

In [8]:
labs.directlake.add_table_to_direct_lake_semantic_model(
    dataset= semanticmodel_name,
    table_name= target_table, # reusing the target table name, as the gold lakehouse should already be self explanatory
    lakehouse_table_name= target_table,
    refresh= False,
    workspace= None # if not specified, will be the same workspace as the notebook runs
)

StatementMeta(, fc4b98e5-69eb-41d3-a6d2-bfbb71cc3aa6, 16, Finished, Available, Finished)

🟢 The 'sales' table has been added to the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'sales' partition has been added to the 'sales' table in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'OrderNumber' column has been added to the 'sales' table as a 'String' data type in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'ProductKey' column has been added to the 'sales' table as a 'Int64' data type in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'OrderDateKey' column has been added to the 'sales' table as a 'Int64' data type in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'ShipDateKey' column has been added to the 'sales' table as a 'Int64' data type in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'Custome

## Add relationships to the model
After we added the tables, we need to create relationships to the dimension tables

In [4]:
# setup connection to Tabular Object Model (TOM) and list tables as example
with labs.tom.connect_semantic_model(dataset=semanticmodel_name, readonly=False, workspace=None) as tom:
    for t in tom.model.Tables:
        print(t.Name)

StatementMeta(, 85c6cecf-e197-4edd-bb46-37d3121e82b8, 12, Finished, Available, Finished)

_measures
product
sales
date


In [5]:
# Define relationships as they should be added to the model
# From should always be the many side of the relationship / fact table (current limitation)

relationships = [
    {'FromTable' : 'sales', 'FromColumn':'ProductKey', 'ToTable':'product',  'ToColumn':'ProductKey'},
    {'FromTable' : 'sales', 'FromColumn':'OrderDateKey', 'ToTable':'date',  'ToColumn':'DateKey'}
]

StatementMeta(, 85c6cecf-e197-4edd-bb46-37d3121e82b8, 13, Finished, Available, Finished)

In [6]:
# List current existing relationships from the semantic model
currentrelationships = fabric.list_relationships(semanticmodel_name)

# Add a new column to store results
currentrelationships["relationship_name"] = ""

# add relationship concatenation in a new column to the dataframe
for idx, row in currentrelationships.iterrows():
        relationship_name = labs.create_relationship_name(
            from_table=row["From Table"],
            from_column=row["From Column"],
            to_table=row["To Table"],
            to_column=row["To Column"]
        )
        currentrelationships.at[idx, "relationship_name"] = relationship_name

# Show the updated DataFrame
display(currentrelationships)

StatementMeta(, 85c6cecf-e197-4edd-bb46-37d3121e82b8, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0a7653df-6315-49b9-a6be-7c3aab1a6a9d)

In [7]:
# Validating if relationship already exists in the model

for relationship in relationships:
    relationship_name = labs.create_relationship_name(
            from_table=relationship["FromTable"],
            from_column=relationship["FromColumn"],
            to_table=relationship["ToTable"],
            to_column=relationship["ToColumn"]
        )
    # returning results of validation
    does_relationship_exist = relationship_name in currentrelationships['relationship_name'].tolist()
    print(f'Relationship: {relationship_name}; Does it exist? {does_relationship_exist}')
    
    # if relationship does not exist, create relationship
    if not does_relationship_exist:
        with labs.tom.connect_semantic_model(dataset=semanticmodel_name, readonly=False, workspace=None) as tom:
            tom.add_relationship(
                from_table= relationship["FromTable"], 
                from_column= relationship["FromColumn"], 
                to_table= relationship["ToTable"], 
                to_column= relationship["ToColumn"],
                from_cardinality= 'Many', # ‘Many’, ‘One’, ‘None’
                to_cardinality = 'One', # ‘Many’, ‘One’, ‘None’
                cross_filtering_behavior= 'OneDirection', # ‘OneDirection’, ‘BothDirections’
                is_active= True, 
                security_filtering_behavior= None, 
                rely_on_referential_integrity= False)
            print(f'✅ Relationship {relationship_name} has been created') 
 
 # Rerunning this cell may result in errors, as the cell above (listing the current relationships) forms an input. This input may be cached and therefore return errors. 
 # If the relationship already exists, the error lists something along the lines of "ambiguous path between .... "

StatementMeta(, 85c6cecf-e197-4edd-bb46-37d3121e82b8, 15, Finished, Available, Finished)

Relationship: 'sales'[ProductKey] -> 'product'[ProductKey]; Does it exist? True
Relationship: 'sales'[OrderDateKey] -> 'date'[DateKey]; Does it exist? False
✅ Relationship 'sales'[OrderDateKey] -> 'date'[DateKey] has been created
